# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

ModuleNotFoundError: ignored

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import pandas as pd
from nltk.tokenize import word_tokenize

recipes = pd.read_csv('recipes_sample.csv').dropna()
words=[]
for des in recipes['description']:
  words+=word_tokenize(des) 
words=list(set(words))

print(words[:5])

['8.5', 'friendly', 'create', 'party/gathering', 'best/traditionally']


1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
from nltk.metrics.distance import *
import random

In [ ]:
listt = ['inferno','magnesium','ufo','dark','moon']
for i in range(5):
  w1,w2 = random.sample(listt,2)
  dist = edit_distance(w1,w2)
  print(f"Distance between '{w1}' and '{w2}' = '{dist}'")

Distance between 'inferno' and 'ufo' = '5'
Distance between 'magnesium' and 'inferno' = '8'
Distance between 'dark' and 'moon' = '4'
Distance between 'dark' and 'inferno' = '6'
Distance between 'inferno' and 'ufo' = '5'


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
def search_old(word):
  words = ['inferno','magnesium','ufo','dark','moon','thelema']
  for j in words:
       dist = int(edit_distance(word,j))
       print(f"Distance between '{word}' and '{j}' = '{dist}'")

In [ ]:
def search(word,k):
  words = ['inferno','magnesium','ufo','dark','moon','thelema']
  dist = {j:edit_distance(word,j) for j in words}
  res = sorted(dist,key = dist.get)[:k]
  return res

In [ ]:
search('tramendous',3)

['inferno', 'magnesium', 'moon']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [5]:
import nltk
from nltk.stem import *
import pandas as pd
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
import pandas as pd
from nltk.tokenize import word_tokenize

recipes = pd.read_csv('recipes_sample.csv').dropna()
words=[]
for des in recipes['description']:
  words+=word_tokenize(des) 
words=list(set(words))

#print(words)

stemma = SnowballStemmer('english')
lemma = WordNetLemmatizer()
#words = ['inferno','magnesium','cats','darkness','moons','thelema']
df = pd.DataFrame(index = [word for word in words],columns=['word','stemmed_word','normalized_word'])
for word in words:
  df.at[word,'word'] = word
  df.at[word,'stemmed_word'] = stemma.stem(word)
  df.at[word,'normalized_word'] = lemma.lemmatize(word)

print(df[:5])

                                  word stemmed_word     normalized_word
8.5                                8.5          8.5                 8.5
friendly                      friendly       friend            friendly
create                          create        creat              create
party/gathering        party/gathering   party/gath     party/gathering
best/traditionally  best/traditionally  best/tradit  best/traditionally


In [ ]:
stemma = SnowballStemmer('english')
lemma = WordNetLemmatizer()
words = ['inferno','magnesium','cats','darkness','moons','thelema']
df = pd.DataFrame(index = [word for word in words],columns=['word','stemmed_word','normalized_word'])
for word in words:
  df.at[word,'word'] = word
  df.at[word,'stemmed_word'] = stemma.stem(word)
  df.at[word,'normalized_word'] = lemma.lemmatize(word)

print(df)



                word stemmed_word normalized_word
inferno      inferno      inferno         inferno
magnesium  magnesium    magnesium       magnesium
cats            cats          cat             cat
darkness    darkness         dark        darkness
moons          moons         moon            moon
thelema      thelema      thelema         thelema


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
import re

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
recipe_c = recipes.copy()
recipe_c = recipe_c.astype({'description':str})
new = recipe_c['description'].tolist()
for i in range(0,len(new)):
  re.sub('\d', ' ',new[i])

In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words = list(set(stopwords.words('english')))
tok = word_tokenize(new.lower())
tok_wto_stopw=[word for word in tok if word not in stop_words]
tok_wto_stopw

In [ ]:
stop_words = list(set(stopwords.words('english')))

filtered_words = [new[i] for i in range(len(new)) if new[i].lower() not in stop_words]
top10_before = pd.Series(new).value_counts().head(10)
top10_after = pd.Series(filtered_words).value_counts().head(10)

stop_wr_count = len([i for i in new if i.lower() in stop_words])
total_wr_count = len(new)
stop_ratio = stop_wr_count / total_wr_count

print('Stop_words_count',stop_wr_count)
print('total_wr_count', total_wr_count)
print('stop_wr_count',stop_wr_count)

Stop_words_count 0
total_wr_count 30000
stop_wr_count 0


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`(важность использования каждого слова из некоторого набора слов (количество слов набора определяет размерность вектора))

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

recipes = pd.read_csv('recipes_sample.csv')
random_recs = recipes.sample(n=5)

vectorz = TfidfVectorizer(stop_words = 'english')
vectors = vectorz.fit_transform(random_recs['description'])

k = 0 
for i,rec in random_recs.iterrows():
  print(f"Recipe {i+1}:{rec['name']}")
  print(f"Vector description : {vectors[k].toarray()}")
  k+=1

Recipe 12124:gluten free   like campbells   tomato soup
Vector description : [[0.         0.         0.20851441 0.20851441 0.20851441 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.20851441 0.         0.         0.20851441
  0.         0.         0.         0.         0.         0.
  0.         0.20851441 0.         0.         0.20851441 0.
  0.         0.         0.         0.         0.20851441 0.
  0.         0.20851441 0.         0.         0.62554324 0.
  0.         0.         0.         0.         0.41702883 0.
  0.20851441 0.         0.         0.        ]]
Recipe 13595:hellman s grilled asian kabobs appetizer  shrimp
Vector description : [[0.26726124 0.26726124 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.26726124 0.         0.26726124 0.26726124 0.         0.
  0.         0.         0.26726124 0.26726124 0.         0.26726

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [8]:
from scipy.spatial import distance

In [40]:
for i in range(vectors.shape[0]): 
  print(vectors[i].toarray())

[[0.         0.         0.20851441 0.20851441 0.20851441 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.20851441 0.         0.         0.20851441
  0.         0.         0.         0.         0.         0.
  0.         0.20851441 0.         0.         0.20851441 0.
  0.         0.         0.         0.         0.20851441 0.
  0.         0.20851441 0.         0.         0.62554324 0.
  0.         0.         0.         0.         0.41702883 0.
  0.20851441 0.         0.         0.        ]]
[[0.26726124 0.26726124 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.26726124 0.         0.26726124 0.26726124 0.         0.
  0.         0.         0.26726124 0.26726124 0.         0.26726124
  0.         0.26726124 0.         0.         0.         0.26726124
  0.26726124 0.         0.         0.         0.         0.26726124
  0.         0.     

In [43]:
import numpy as np

In [44]:
def _validate_vector(u, dtype=None):
    # XXX Is order='c' really necessary?
    u = np.asarray(u, dtype=dtype, order='c')
    if u.ndim == 1:
        return u
    raise ValueError("Input vector should be 1-D.")

In [45]:
vectors[0].toarray().ndim

2

In [48]:
si = [] 
for i in range(vectors.shape[0]): 
  row = []
  for j in range(vectors.shape[0]):
    sim= 1 - distance.cosine(vectors[i].toarray().ravel(), vectors[j].toarray().ravel())   #ravel()-to change the dimension of vector to 1-D
    row.append(sim) 
  si.append(row) 
s_df = pd.DataFrame(si, columns=random_recs['description']) 
print(s_df)

description  this recipe is as easy as opening a can of commercial tomato soup! the flavour is the about the same and everyone loves it! it can be used just as soup or in casseroles calling for tomato soup.  \
0                                                          1.0                                                                                                                                                   
1                                                          0.0                                                                                                                                                   
2                                                          0.0                                                                                                                                                   
3                                                          0.0                                                                                                  